In [ ]:
import os

import numpy as np
import pycolmap
import matplotlib.pyplot as plt

from megadepth.utils.io import load_depth_map, load_image

In [ ]:
base_dir = os.path.join("..", "data", "scenes")
scene = "0229"
model_name = "superpoint_max-superglue-netvlad-50-KA+BA"

model = pycolmap.Reconstruction(os.path.join(base_dir, scene, "sparse", model_name, "refined"))
print(model.summary())

In [ ]:
def plot_depth(img_id):
    img_name = model.images[img_id].name

    img = load_image(os.path.join(base_dir, scene, "images", img_name))

    depth = load_depth_map(
        os.path.join(
            base_dir,
            scene,
            "dense",
            model_name,
            "stereo",
            "depth_maps",
            f"{img_name}.geometric.bin",
        )
    )

    depth = np.where(depth > 0, depth, np.nan)

    d_min = np.nanmin(depth)
    d_max = np.nanmax(depth)

    # normalize depth
    depth = (depth - d_min) / (d_max - d_min)

    fig, ax = plt.subplots(1, 3, figsize=(20, 10))

    ax[0].imshow(img)
    ax[0].set_title("Image")

    ax[1].imshow(depth)
    ax[1].set_title("Depth")

    ax[2].imshow(img)
    ax[2].imshow(depth, alpha=0.5)
    ax[2].set_title("Image + Depth")

    plt.show()

In [ ]:
rand_img_ids = np.random.choice(list(model.images.keys()), 5)

for img_id in rand_img_ids:
    plot_depth(img_id)

In [ ]:
completeness = []

for img_id in model.images.keys():
    img_name = model.images[img_id].name

    depth = load_depth_map(
        os.path.join(
            base_dir,
            scene,
            "dense",
            model_name,
            "stereo",
            "depth_maps",
            f"{img_name}.geometric.bin",
        )
    )

    depth = np.where(depth > 0, depth, np.nan)

    score = np.count_nonzero(~np.isnan(depth)) / depth.size

    assert type(score) == float
    completeness.append((img_id, score))
    

print(f"Mean Completness: {np.mean([c[1] for c in completeness])}")

In [ ]:
plt.hist([c[1] for c in completeness], bins=20)
plt.title("Completness")

plt.show()

In [ ]:
# number will be even bigger as we should discard the sky from the calculation
n_valid = np.sum([c[1] > 0.3 for c in completeness]) 
print(f"Number of images with completeness > 0.3: {n_valid}")

In [ ]:
# plot 5 image with lowest completeness
for img_id, _ in sorted(completeness, key=lambda x: x[1])[:5]:
    plot_depth(img_id)

In [ ]:
# plot 5 image with highest completeness
for img_id, _ in sorted(completeness, key=lambda x: x[1], reverse=True)[:5]:
    plot_depth(img_id)

In [ ]:
def plot_normal(img_id):
    img_name = model.images[img_id].name

    img = load_image(os.path.join(base_dir, scene, "images", img_name))

    normal = load_depth_map(
        os.path.join(
            base_dir,
            scene,
            "dense",
            model_name,
            "stereo",
            "normal_maps",
            f"{img_name}.geometric.bin",
        )
    )

    # normalize to 0-1
    normal = (normal - normal.min()) / (normal.max() - normal.min())

    fig, ax = plt.subplots(1, 3, figsize=(20, 10))

    ax[0].imshow(img)
    ax[0].set_title("Image")

    ax[1].imshow(normal)
    ax[1].set_title("Normal")

    ax[2].imshow(img)
    ax[2].imshow(normal, alpha=0.5)
    ax[2].set_title("Image + Normal")

    plt.show()

In [ ]:
# plot 5 image with highest completeness
for img_id, _ in sorted(completeness, key=lambda x: x[1], reverse=True)[:5]:
    plot_normal(img_id)